In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
import numpy as np 
import pandas as pd 
import seaborn as sns
!pip install gensim spacy nltk
import re 
import gensim
import spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
data= pd.read_csv('/content/drive/My Drive/Bachelor/Cleaned Data/Sarcasm.csv') 

In [6]:
import numpy as np
max_sequence_length = np.max(data['Number of Words in tweet'])
print("max length is:", max_sequence_length)

max length is: 56


In [7]:
text = data['cleaned_tweet']
label = data['sarcasm']

In [8]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(text.astype(str))
sequences = tokenizer.texts_to_sequences(text.astype(str))

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))


from keras_preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Found 55395 unique tokens.
Shape of data tensor: (26095, 56)


In [9]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(label))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (26095, 2)


In [10]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

# Arabert

In [11]:
!pip install bert-for-tf2
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=008b93d7316ecacfb05cc17c624de69ae1637a5061352a62f62641e4856846bd
  Stored in directory: /root/.cache/pip/wheels/ab/a4/72/df07592cea3ae06b5e846f5e52262f8b16748e829ca354b7df
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=9efcb7130f4a1d2fe7a9c34f6feb758018cd6b16c7a89b262f0d371efd15727f
  Stored in directory: /root/.cache/pip/wheels/c7/f3/85/b8cf1d8bfe55dc2ece0f1fcd4e91d6f8fc7b59ff3fd75329e1
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=579a6c78a3271bfafa89c2b53144c3384add9c6704fafdc00b6707c1d0e657fc
  Sto

In [12]:
import numpy as np
import math
import re
import pandas as pd
import random

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [13]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
from transformers import AutoTokenizer, AutoModel

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [14]:
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv01",do_lower_case=False)
def encode_sentence(sent):
    tokenized = arabert_tokenizer.tokenize(sent)
    return arabert_tokenizer.convert_tokens_to_ids(tokenized)

Downloading:   0%|          | 0.00/379 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [15]:
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

In [16]:
from keras import backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# GRU-BiLSTM

In [ ]:
# GRU layer followed by BiLSTM layer
from tensorflow.keras import layers
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 17s 20ms/step - loss: 0.4715 - accuracy: 0.8203
Epoch 2/10
612/612 [==============================] - 14s 23ms/step - loss: 0.2764 - accuracy: 0.8849
Epoch 3/10
612/612 [==============================] - 11s 19ms/step - loss: 0.1423 - accuracy: 0.9496
Epoch 4/10
612/612 [==============================] - 14s 22ms/step - loss: 0.0860 - accuracy: 0.9722
Epoch 5/10
612/612 [==============================] - 14s 23ms/step - loss: 0.0574 - accuracy: 0.9826
Epoch 6/10
612/612 [==============================] - 14s 22ms/step - loss: 0.0429 - accuracy: 0.9874
Epoch 7/10
612/612 [==============================] - 14s 24ms/step - loss: 0.0352 - accuracy: 0.9899
Epoch 8/10
612/612 [==============================] - 14s 23ms/step - loss: 0.0312 - accuracy: 0.9909
Epoch 9/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0268 - accuracy: 0.9921
Epoch 10/10
612/612 [==============================] - 14s 22ms/step - loss: 0.0258 - accurac

In [ ]:
predictions= model.predict(x_test)

predictions=np.argmax(predictions, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions))
cm = confusion_matrix(y_test, predictions)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions))
print('F1 Score: %.3f' % f1_score(y_test, predictions))
print('Precision: %.3f' % precision_score(y_test, predictions))

204/204 [==============================] - 2s 5ms/step
Recall: 0.905
The Confusion Matrix is [[5306   75]
 [ 109 1033]]
Accuracy: 0.972
F1 Score: 0.918
Precision: 0.932


In [ ]:
#GRU - BiLSTM, adding a flatten layer

from tensorflow.keras import layers
model1= tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model1.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 20s 23ms/step - loss: 0.4761 - accuracy: 0.8171
Epoch 2/10
612/612 [==============================] - 13s 22ms/step - loss: 0.2749 - accuracy: 0.8890
Epoch 3/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1387 - accuracy: 0.9504
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0794 - accuracy: 0.9750
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0509 - accuracy: 0.9847
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0406 - accuracy: 0.9884
Epoch 7/10
612/612 [==============================] - 11s 19ms/step - loss: 0.0338 - accuracy: 0.9905
Epoch 8/10
612/612 [==============================] - 11s 19ms/step - loss: 0.0304 - accuracy: 0.9911
Epoch 9/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0260 - accuracy: 0.9920
Epoch 10/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0247 - accurac

In [ ]:
predictions1= model1.predict(x_test)

predictions1=np.argmax(predictions1, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions1))
cm = confusion_matrix(y_test, predictions1)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions1))
print('F1 Score: %.3f' % f1_score(y_test, predictions1))
print('Precision: %.3f' % precision_score(y_test, predictions1))

204/204 [==============================] - 2s 5ms/step
Recall: 0.903
The confusion matrix is [[5305   70]
 [ 111 1037]]
Accuracy: 0.972
F1 Score: 0.920
Precision: 0.937


In [ ]:
#GRU - BiLSTM with a flatten layer and also a Softmax activation function for classification, but with a RMSprop optimizer

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-4),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 18s 19ms/step - loss: 0.4796 - accuracy: 0.8182
Epoch 2/10
612/612 [==============================] - 12s 20ms/step - loss: 0.4074 - accuracy: 0.8265
Epoch 3/10
612/612 [==============================] - 14s 24ms/step - loss: 0.3426 - accuracy: 0.8551
Epoch 4/10
612/612 [==============================] - 10s 17ms/step - loss: 0.3116 - accuracy: 0.8710
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2865 - accuracy: 0.8845
Epoch 6/10
612/612 [==============================] - 9s 15ms/step - loss: 0.2643 - accuracy: 0.8964
Epoch 7/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2429 - accuracy: 0.9060
Epoch 8/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2230 - accuracy: 0.9163
Epoch 9/10
612/612 [==============================] - 8s 14ms/step - loss: 0.2025 - accuracy: 0.9250
Epoch 10/10
612/612 [==============================] - 8s 14ms/step - loss: 0.1837 - accuracy: 0.

In [ ]:
predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

204/204 [==============================] - 2s 5ms/step
Recall: 0.719
The Confusion Matrix is [[5205  173]
 [ 322  823]]
Accuracy: 0.924
F1 Score: 0.769
Precision: 0.826


#Learning Rate & Optimizers Tuning

In [ ]:
# GRU - BiLSTM, Adam(1e-4)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model6 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model6.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',get_f1])

model6.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy6= model6.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy6)

predictions6= model6.predict(x_test)

predictions6=np.argmax(predictions6, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions6))
cm = confusion_matrix(y_test, predictions6)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions6))
print('F1 Score: %.3f' % f1_score(y_test, predictions6))
print('Precision: %.3f' % precision_score(y_test, predictions6))

Epoch 1/10
612/612 [==============================] - 15s 18ms/step - loss: 0.4703 - accuracy: 0.8190 - get_f1: 0.8190
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2736 - accuracy: 0.8856 - get_f1: 0.8855
Epoch 3/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1403 - accuracy: 0.9492 - get_f1: 0.9492
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0818 - accuracy: 0.9733 - get_f1: 0.9733
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0537 - accuracy: 0.9847 - get_f1: 0.9847
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0405 - accuracy: 0.9887 - get_f1: 0.9887
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0342 - accuracy: 0.9897 - get_f1: 0.9897
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0296 - accuracy: 0.9915 - get_f1: 0.9915
Epoch 9/10
612/612 [============================

In [ ]:
#Model 7: 1 GRU - BiLSTM , Adam 1e-3

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model7 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model7.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy',get_f1])

model7.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy7= model7.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy7)

predictions7= model7.predict(x_test)

predictions7=np.argmax(predictions7, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions7))
cm = confusion_matrix(y_test, predictions7)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions7))
print('F1 Score: %.3f' % f1_score(y_test, predictions7))
print('Precision: %.3f' % precision_score(y_test, predictions7))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 15s 18ms/step - loss: 0.3744 - accuracy: 0.8480 - get_f1: 0.8480
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1460 - accuracy: 0.9487 - get_f1: 0.9487
Epoch 3/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0500 - accuracy: 0.9848 - get_f1: 0.9848
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0222 - accuracy: 0.9922 - get_f1: 0.9922
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0141 - accuracy: 0.9932 - get_f1: 0.9932
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0103 - accuracy: 0.9942 - get_f1: 0.9942
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0113 - accuracy: 0.9937 - get_f1: 0.9937
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0118 - accuracy: 0.9936 - get_f1: 0.9936
Epoch 9/10
612/612 [==============================] - 11s 1

In [ ]:
#Model 7: 1 GRU - BiLSTM , Adam 1e-2

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model8 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model8.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-2),
              metrics=['accuracy',get_f1])

model8.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy8= model8.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy8)

predictions8= model8.predict(x_test)

predictions8=np.argmax(predictions8, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions8))
cm = confusion_matrix(y_test, predictions8)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions8))
print('F1 Score: %.3f' % f1_score(y_test, predictions8))
print('Precision: %.3f' % precision_score(y_test, predictions8))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 15s 18ms/step - loss: 0.3842 - accuracy: 0.8413 - get_f1: 0.8413
Epoch 2/10
612/612 [==============================] - 12s 19ms/step - loss: 0.2029 - accuracy: 0.9194 - get_f1: 0.9194
Epoch 3/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1399 - accuracy: 0.9490 - get_f1: 0.9490
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1041 - accuracy: 0.9621 - get_f1: 0.9621
Epoch 5/10
612/612 [==============================] - 11s 17ms/step - loss: 0.0837 - accuracy: 0.9702 - get_f1: 0.9701
Epoch 6/10
612/612 [==============================] - 11s 17ms/step - loss: 0.0670 - accuracy: 0.9753 - get_f1: 0.9753
Epoch 7/10
612/612 [==============================] - 11s 17ms/step - loss: 0.0584 - accuracy: 0.9788 - get_f1: 0.9788
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0520 - accuracy: 0.9807 - get_f1: 0.9807
Epoch 9/10
612/612 [==============================] - 11s 1

In [ ]:
#Model 7: 1 GRU - BiLSTM , Adam 1e-1

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model9 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model9.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-1),
              metrics=['accuracy',get_f1])

model9.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy9= model9.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy9)

predictions9= model9.predict(x_test)

predictions9=np.argmax(predictions9, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions9))
cm = confusion_matrix(y_test, predictions9)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions9))
print('F1 Score: %.3f' % f1_score(y_test, predictions9))
print('Precision: %.3f' % precision_score(y_test, predictions9))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 15s 18ms/step - loss: 0.5279 - accuracy: 0.8159 - get_f1: 0.8158
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4750 - accuracy: 0.8185 - get_f1: 0.8185
Epoch 3/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4755 - accuracy: 0.8185 - get_f1: 0.8184
Epoch 4/10
612/612 [==============================] - 11s 17ms/step - loss: 0.4753 - accuracy: 0.8184 - get_f1: 0.8183
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4753 - accuracy: 0.8185 - get_f1: 0.8184
Epoch 6/10
612/612 [==============================] - 12s 19ms/step - loss: 0.4756 - accuracy: 0.8185 - get_f1: 0.8185
Epoch 7/10
612/612 [==============================] - 11s 17ms/step - loss: 0.4753 - accuracy: 0.8185 - get_f1: 0.8185
Epoch 8/10
612/612 [==============================] - 11s 17ms/step - loss: 0.4756 - accuracy: 0.8185 - get_f1: 0.8184
Epoch 9/10
612/612 [==============================] - 12s 1

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Model 7: 1 GRU - BiLSTM , RMSprop 1e-4

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model10 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model10.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-4),
              metrics=['accuracy',get_f1])

model10.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy10= model10.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy10)

predictions10= model10.predict(x_test)

predictions10=np.argmax(predictions10, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions10))
cm = confusion_matrix(y_test, predictions10)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions10))
print('F1 Score: %.3f' % f1_score(y_test, predictions10))
print('Precision: %.3f' % precision_score(y_test, predictions10))

Epoch 1/10
612/612 [==============================] - 14s 15ms/step - loss: 0.4794 - accuracy: 0.8192 - get_f1: 0.8192
Epoch 2/10
612/612 [==============================] - 9s 15ms/step - loss: 0.4001 - accuracy: 0.8253 - get_f1: 0.8253
Epoch 3/10
612/612 [==============================] - 9s 14ms/step - loss: 0.3418 - accuracy: 0.8565 - get_f1: 0.8565
Epoch 4/10
612/612 [==============================] - 9s 14ms/step - loss: 0.3126 - accuracy: 0.8735 - get_f1: 0.8736
Epoch 5/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2876 - accuracy: 0.8841 - get_f1: 0.8841
Epoch 6/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2670 - accuracy: 0.8948 - get_f1: 0.8948
Epoch 7/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2470 - accuracy: 0.9053 - get_f1: 0.9053
Epoch 8/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2293 - accuracy: 0.9132 - get_f1: 0.9132
Epoch 9/10
612/612 [==============================] - 9

In [ ]:
#Model 7: 1 GRU - BiLSTM , RMSprop 1e-3

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model11 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model11.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-3),
              metrics=['accuracy',get_f1])

model11.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy11= model11.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy11)

predictions11= model11.predict(x_test)

predictions11=np.argmax(predictions11, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions11))
cm = confusion_matrix(y_test, predictions11)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions11))
print('F1 Score: %.3f' % f1_score(y_test, predictions11))
print('Precision: %.3f' % precision_score(y_test, predictions11))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 13s 14ms/step - loss: 0.3982 - accuracy: 0.8362 - get_f1: 0.8363
Epoch 2/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2537 - accuracy: 0.9061 - get_f1: 0.9061
Epoch 3/10
612/612 [==============================] - 9s 14ms/step - loss: 0.1728 - accuracy: 0.9377 - get_f1: 0.9376
Epoch 4/10
612/612 [==============================] - 10s 17ms/step - loss: 0.1228 - accuracy: 0.9575 - get_f1: 0.9575
Epoch 5/10
612/612 [==============================] - 10s 16ms/step - loss: 0.0922 - accuracy: 0.9679 - get_f1: 0.9679
Epoch 6/10
612/612 [==============================] - 9s 14ms/step - loss: 0.0701 - accuracy: 0.9767 - get_f1: 0.9767
Epoch 7/10
612/612 [==============================] - 9s 14ms/step - loss: 0.0543 - accuracy: 0.9829 - get_f1: 0.9829
Epoch 8/10
612/612 [==============================] - 9s 14ms/step - loss: 0.0440 - accuracy: 0.9862 - get_f1: 0.9862
Epoch 9/10
612/612 [==============================] - 9s 14ms/st

In [ ]:
#Model 7: 1 GRU - BiLSTM , RMSprop 1e-2

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model12 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model12.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-2),
              metrics=['accuracy',get_f1])

model12.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy12= model12.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy12)

predictions12= model12.predict(x_test)

predictions12=np.argmax(predictions12, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions12))
cm = confusion_matrix(y_test, predictions12)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions12))
print('F1 Score: %.3f' % f1_score(y_test, predictions12))
print('Precision: %.3f' % precision_score(y_test, predictions12))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 14s 15ms/step - loss: 0.3952 - accuracy: 0.8427 - get_f1: 0.8428
Epoch 2/10
612/612 [==============================] - 9s 14ms/step - loss: 0.1977 - accuracy: 0.9316 - get_f1: 0.9316
Epoch 3/10
612/612 [==============================] - 9s 14ms/step - loss: 0.1213 - accuracy: 0.9641 - get_f1: 0.9641
Epoch 4/10
612/612 [==============================] - 9s 14ms/step - loss: 0.0896 - accuracy: 0.9753 - get_f1: 0.9753
Epoch 5/10
612/612 [==============================] - 9s 14ms/step - loss: 0.0697 - accuracy: 0.9814 - get_f1: 0.9814
Epoch 6/10
612/612 [==============================] - 9s 14ms/step - loss: 0.0626 - accuracy: 0.9848 - get_f1: 0.9848
Epoch 7/10
612/612 [==============================] - 9s 14ms/step - loss: 0.0576 - accuracy: 0.9857 - get_f1: 0.9857
Epoch 8/10
612/612 [==============================] - 9s 14ms/step - loss: 0.0523 - accuracy: 0.9889 - get_f1: 0.9889
Epoch 9/10
612/612 [==============================] - 9s 14ms/step

In [ ]:
#Model 7: 1 GRU - BiLSTM , RMSprop 1e-1

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model13 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model13.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-1),
              metrics=['accuracy',get_f1])

model13.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy13= model13.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy13)

predictions13= model13.predict(x_test)

predictions13=np.argmax(predictions13, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions13))
cm = confusion_matrix(y_test, predictions13)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions13))
print('F1 Score: %.3f' % f1_score(y_test, predictions13))
print('Precision: %.3f' % precision_score(y_test, predictions13))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 14s 14ms/step - loss: 0.8879 - accuracy: 0.8011 - get_f1: 0.8011
Epoch 2/10
612/612 [==============================] - 9s 14ms/step - loss: 0.5118 - accuracy: 0.8168 - get_f1: 0.8168
Epoch 3/10
612/612 [==============================] - 9s 14ms/step - loss: 0.5158 - accuracy: 0.8157 - get_f1: 0.8157
Epoch 4/10
612/612 [==============================] - 9s 14ms/step - loss: 0.5169 - accuracy: 0.8180 - get_f1: 0.8179
Epoch 5/10
612/612 [==============================] - 9s 14ms/step - loss: 0.5075 - accuracy: 0.8186 - get_f1: 0.8187
Epoch 6/10
612/612 [==============================] - 9s 14ms/step - loss: 0.5136 - accuracy: 0.8183 - get_f1: 0.8183
Epoch 7/10
612/612 [==============================] - 9s 14ms/step - loss: 0.5601 - accuracy: 0.8158 - get_f1: 0.8157
Epoch 8/10
612/612 [==============================] - 9s 14ms/step - loss: 0.5252 - accuracy: 0.8180 - get_f1: 0.8179
Epoch 9/10
612/612 [==============================] - 9s 15ms/step

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Model 8: 1 GRU - BiLSTM / flatten

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model7 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model7.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',get_f1])

model7.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 19s 22ms/step - loss: 0.4805 - accuracy: 0.8139 - get_f1: 0.8138
Epoch 2/10
612/612 [==============================] - 13s 21ms/step - loss: 0.2780 - accuracy: 0.8877 - get_f1: 0.8877
Epoch 3/10
612/612 [==============================] - 14s 22ms/step - loss: 0.1419 - accuracy: 0.9497 - get_f1: 0.9497
Epoch 4/10
612/612 [==============================] - 15s 25ms/step - loss: 0.0820 - accuracy: 0.9738 - get_f1: 0.9739
Epoch 5/10
612/612 [==============================] - 11s 19ms/step - loss: 0.0532 - accuracy: 0.9838 - get_f1: 0.9837
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0409 - accuracy: 0.9881 - get_f1: 0.9881
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0340 - accuracy: 0.9901 - get_f1: 0.9901
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0300 - accuracy: 0.9916 - get_f1: 0.9916
Epoch 9/10
612/612 [==============================] - 11s 1

In [ ]:
accuracy7= model7.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy7)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


204/204 [==============================] - 2s 6ms/step - loss: 0.1348 - accuracy: 0.9701 - get_f1: 0.9701
test accuracy is:  [0.13484889268875122, 0.970105767250061, 0.9701003432273865]


In [ ]:
predictions7= model7.predict(x_test)

predictions7=np.argmax(predictions7, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions7))
cm = confusion_matrix(y_test, predictions7)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions7))
print('F1 Score: %.3f' % f1_score(y_test, predictions7))
print('Precision: %.3f' % precision_score(y_test, predictions7))

204/204 [==============================] - 2s 5ms/step
Recall: 0.916
The confusion matric is: [[5210   92]
 [ 103 1118]]
Accuracy: 0.970
F1 Score: 0.920
Precision: 0.924


In [ ]:
#Model 9: 1 GRU - BiLSTM / flatten

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model8 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model8.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-4),
              metrics=['accuracy',get_f1])

model8.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 19s 20ms/step - loss: 0.4752 - accuracy: 0.8228 - get_f1: 0.8228
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.4032 - accuracy: 0.8273 - get_f1: 0.8272
Epoch 3/10
612/612 [==============================] - 9s 14ms/step - loss: 0.3441 - accuracy: 0.8531 - get_f1: 0.8531
Epoch 4/10
612/612 [==============================] - 9s 14ms/step - loss: 0.3157 - accuracy: 0.8698 - get_f1: 0.8698
Epoch 5/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2925 - accuracy: 0.8817 - get_f1: 0.8817
Epoch 6/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2717 - accuracy: 0.8924 - get_f1: 0.8924
Epoch 7/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2501 - accuracy: 0.9030 - get_f1: 0.9030
Epoch 8/10
612/612 [==============================] - 9s 14ms/step - loss: 0.2320 - accuracy: 0.9126 - get_f1: 0.9126
Epoch 9/10
612/612 [==============================] - 9s 14ms/ste

In [ ]:
accuracy8= model8.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy8)

204/204 [==============================] - 9s 7ms/step - loss: 0.2303 - accuracy: 0.9169 - get_f1: 0.9170
test accuracy is:  [0.2302737683057785, 0.916909396648407, 0.9169730544090271]


In [ ]:
predictions8= model8.predict(x_test)

predictions8=np.argmax(predictions8, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions8))
cm = confusion_matrix(y_test, predictions8)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions8))
print('F1 Score: %.3f' % f1_score(y_test, predictions8))
print('Precision: %.3f' % precision_score(y_test, predictions8))

204/204 [==============================] - 2s 5ms/step
Recall: 0.702
The confusion matric is: [[5158  192]
 [ 350  823]]
Accuracy: 0.917
F1 Score: 0.752
Precision: 0.811


In [17]:
# GRU - BiLSTM, Nadam(1e-4)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model26 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model26.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Nadam(1e-4),
              metrics=['accuracy',get_f1])

model26.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy26= model26.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy26)

predictions26= model26.predict(x_test)

predictions26=np.argmax(predictions26, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions26))
cm = confusion_matrix(y_test, predictions26)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions26))
print('F1 Score: %.3f' % f1_score(y_test, predictions26))
print('Precision: %.3f' % precision_score(y_test, predictions26))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 27s 25ms/step - loss: 0.4759 - accuracy: 0.8209 - get_f1: 0.8209
Epoch 2/10
612/612 [==============================] - 15s 25ms/step - loss: 0.3244 - accuracy: 0.8614 - get_f1: 0.8613
Epoch 3/10
612/612 [==============================] - 15s 25ms/step - loss: 0.1836 - accuracy: 0.9306 - get_f1: 0.9307
Epoch 4/10
612/612 [==============================] - 16s 26ms/step - loss: 0.1157 - accuracy: 0.9592 - get_f1: 0.9592
Epoch 5/10
612/612 [==============================] - 16s 26ms/step - loss: 0.0797 - accuracy: 0.9739 - get_f1: 0.9740
Epoch 6/10
612/612 [==============================] - 14s 23ms/step - loss: 0.0575 - accuracy: 0.9816 - get_f1: 0.9816
Epoch 7/10
612/612 [==============================] - 16s 27ms/step - loss: 0.0444 - accuracy: 0.9867 - get_f1: 0.9867
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0372 - accuracy: 0.9884 - get_f1: 0.9884
Epoch 9/10
612/612 [==============================] - 15s 2

In [18]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [19]:
# GRU - BiLSTM, Adagrad(1e-4)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model27 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model27.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adagrad(1e-4),
              metrics=['accuracy',get_f1])

model27.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy27= model27.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy27)

predictions27= model27.predict(x_test)

predictions27=np.argmax(predictions27, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions27))
cm = confusion_matrix(y_test, predictions27)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions27))
print('F1 Score: %.3f' % f1_score(y_test, predictions27))
print('Precision: %.3f' % precision_score(y_test, predictions27))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 11s 12ms/step - loss: 0.6854 - accuracy: 0.7880 - get_f1: 0.7880
Epoch 2/10
612/612 [==============================] - 7s 12ms/step - loss: 0.6765 - accuracy: 0.8193 - get_f1: 0.8194
Epoch 3/10
612/612 [==============================] - 7s 12ms/step - loss: 0.6696 - accuracy: 0.8194 - get_f1: 0.8195
Epoch 4/10
612/612 [==============================] - 10s 16ms/step - loss: 0.6632 - accuracy: 0.8194 - get_f1: 0.8192
Epoch 5/10
612/612 [==============================] - 12s 20ms/step - loss: 0.6571 - accuracy: 0.8194 - get_f1: 0.8194
Epoch 6/10
612/612 [==============================] - 11s 19ms/step - loss: 0.6512 - accuracy: 0.8194 - get_f1: 0.8193
Epoch 7/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6455 - accuracy: 0.8194 - get_f1: 0.8195
Epoch 8/10
612/612 [==============================] - 7s 12ms/step - loss: 0.6402 - accuracy: 0.8194 - get_f1: 0.8194
Epoch 9/10
612/612 [==============================] - 7s 12ms/s

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [21]:
# GRU - BiLSTM, Adadelta(1e-4)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model28 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model28.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adadelta(1e-4),
              metrics=['accuracy',get_f1])

model28.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy28= model28.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy28)

predictions28= model28.predict(x_test)

predictions28=np.argmax(predictions28, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions28))
cm = confusion_matrix(y_test, predictions28)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions28))
print('F1 Score: %.3f' % f1_score(y_test, predictions28))
print('Precision: %.3f' % precision_score(y_test, predictions28))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 12s 12ms/step - loss: 0.6957 - accuracy: 0.2690 - get_f1: 0.2690
Epoch 2/10
612/612 [==============================] - 7s 12ms/step - loss: 0.6951 - accuracy: 0.3111 - get_f1: 0.3111
Epoch 3/10
612/612 [==============================] - 7s 12ms/step - loss: 0.6944 - accuracy: 0.3655 - get_f1: 0.3656
Epoch 4/10
612/612 [==============================] - 7s 12ms/step - loss: 0.6938 - accuracy: 0.4256 - get_f1: 0.4257
Epoch 5/10
612/612 [==============================] - 7s 12ms/step - loss: 0.6931 - accuracy: 0.5058 - get_f1: 0.5058
Epoch 6/10
612/612 [==============================] - 8s 14ms/step - loss: 0.6925 - accuracy: 0.5791 - get_f1: 0.5792
Epoch 7/10
612/612 [==============================] - 7s 12ms/step - loss: 0.6918 - accuracy: 0.6517 - get_f1: 0.6518
Epoch 8/10
612/612 [==============================] - 8s 12ms/step - loss: 0.6911 - accuracy: 0.7053 - get_f1: 0.7054
Epoch 9/10
612/612 [==============================] - 7s 12ms/step

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [23]:
# GRU - BiLSTM, Adamax(1e-4)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model29 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model29.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adamax(1e-4),
              metrics=['accuracy',get_f1])

model29.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy29= model29.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy29)

predictions29= model29.predict(x_test)

predictions29=np.argmax(predictions29, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions29))
cm = confusion_matrix(y_test, predictions29)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions29))
print('F1 Score: %.3f' % f1_score(y_test, predictions29))
print('Precision: %.3f' % precision_score(y_test, predictions29))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 12s 12ms/step - loss: 0.5063 - accuracy: 0.8198 - get_f1: 0.8198
Epoch 2/10
612/612 [==============================] - 8s 12ms/step - loss: 0.4658 - accuracy: 0.8206 - get_f1: 0.8206
Epoch 3/10
612/612 [==============================] - 8s 12ms/step - loss: 0.4598 - accuracy: 0.8206 - get_f1: 0.8205
Epoch 4/10
612/612 [==============================] - 8s 12ms/step - loss: 0.4513 - accuracy: 0.8206 - get_f1: 0.8206
Epoch 5/10
612/612 [==============================] - 8s 12ms/step - loss: 0.4386 - accuracy: 0.8206 - get_f1: 0.8206
Epoch 6/10
612/612 [==============================] - 8s 12ms/step - loss: 0.4165 - accuracy: 0.8231 - get_f1: 0.8230
Epoch 7/10
612/612 [==============================] - 8s 12ms/step - loss: 0.3852 - accuracy: 0.8299 - get_f1: 0.8299
Epoch 8/10
612/612 [==============================] - 8s 13ms/step - loss: 0.3561 - accuracy: 0.8419 - get_f1: 0.8419
Epoch 9/10
612/612 [==============================] - 9s 14ms/step

# Activation Function Tuning 

In [ ]:
#GRU - BiLSTM , Adam 1e-4, (Softplus + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model13 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='softplus'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model13.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',get_f1])

model13.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy13= model13.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy13)

predictions13= model13.predict(x_test)

predictions13=np.argmax(predictions13, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions13)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions13))
print('Recall: %.3f' % recall_score(y_test, predictions13))
print('Precision: %.3f' % precision_score(y_test, predictions13))
print('F1 Score: %.3f' % f1_score(y_test, predictions13))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 21s 18ms/step - loss: 0.5295 - accuracy: 0.7895 - get_f1: 0.7896
Epoch 2/10
612/612 [==============================] - 11s 17ms/step - loss: 0.3146 - accuracy: 0.8731 - get_f1: 0.8731
Epoch 3/10
612/612 [==============================] - 11s 17ms/step - loss: 0.1653 - accuracy: 0.9432 - get_f1: 0.9432
Epoch 4/10
612/612 [==============================] - 12s 20ms/step - loss: 0.0992 - accuracy: 0.9679 - get_f1: 0.9679
Epoch 5/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0647 - accuracy: 0.9816 - get_f1: 0.9816
Epoch 6/10
612/612 [==============================] - 11s 17ms/step - loss: 0.0485 - accuracy: 0.9864 - get_f1: 0.9864
Epoch 7/10
612/612 [==============================] - 11s 17ms/step - loss: 0.0385 - accuracy: 0.9893 - get_f1: 0.9893
Epoch 8/10
612/612 [==============================] - 14s 23ms/step - loss: 0.0328 - accuracy: 0.9911 - get_f1: 0.9911
Epoch 9/10
612/612 [==============================] - 16s 2

In [ ]:
#GRU - BiLSTM , Adam 1e-4, (softsign + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model14 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='softsign'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model14.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',get_f1])

model14.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy14= model14.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy14)

predictions14= model14.predict(x_test)

predictions14=np.argmax(predictions14, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions14)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions14))
print('Recall: %.3f' % recall_score(y_test, predictions14))
print('Precision: %.3f' % precision_score(y_test, predictions14))
print('F1 Score: %.3f' % f1_score(y_test, predictions14))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 16s 19ms/step - loss: 0.4659 - accuracy: 0.8168 - get_f1: 0.8169
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2591 - accuracy: 0.8946 - get_f1: 0.8946
Epoch 3/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1319 - accuracy: 0.9523 - get_f1: 0.9523
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0767 - accuracy: 0.9755 - get_f1: 0.9755
Epoch 5/10
612/612 [==============================] - 14s 23ms/step - loss: 0.0526 - accuracy: 0.9844 - get_f1: 0.9844
Epoch 6/10
612/612 [==============================] - 17s 28ms/step - loss: 0.0399 - accuracy: 0.9878 - get_f1: 0.9878
Epoch 7/10
612/612 [==============================] - 18s 30ms/step - loss: 0.0309 - accuracy: 0.9910 - get_f1: 0.9910
Epoch 8/10
612/612 [==============================] - 14s 24ms/step - loss: 0.0304 - accuracy: 0.9912 - get_f1: 0.9912
Epoch 9/10
612/612 [==============================] - 11s 1

In [ ]:
#GRU - BiLSTM , Adam 1e-4, (tanh + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model15 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model15.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',get_f1])

model15.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy15= model15.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy15)

predictions15= model15.predict(x_test)

predictions15=np.argmax(predictions15, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions15)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions15))
print('Recall: %.3f' % recall_score(y_test, predictions15))
print('Precision: %.3f' % precision_score(y_test, predictions15))
print('F1 Score: %.3f' % f1_score(y_test, predictions15))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 15s 18ms/step - loss: 0.4627 - accuracy: 0.8165 - get_f1: 0.8165
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2556 - accuracy: 0.8947 - get_f1: 0.8948
Epoch 3/10
612/612 [==============================] - 11s 19ms/step - loss: 0.1271 - accuracy: 0.9539 - get_f1: 0.9539
Epoch 4/10
612/612 [==============================] - 14s 23ms/step - loss: 0.0712 - accuracy: 0.9764 - get_f1: 0.9764
Epoch 5/10
612/612 [==============================] - 16s 25ms/step - loss: 0.0451 - accuracy: 0.9859 - get_f1: 0.9860
Epoch 6/10
612/612 [==============================] - 12s 20ms/step - loss: 0.0349 - accuracy: 0.9892 - get_f1: 0.9892
Epoch 7/10
612/612 [==============================] - 12s 19ms/step - loss: 0.0274 - accuracy: 0.9914 - get_f1: 0.9914
Epoch 8/10
612/612 [==============================] - 16s 26ms/step - loss: 0.0248 - accuracy: 0.9926 - get_f1: 0.9926
Epoch 9/10
612/612 [==============================] - 15s 2

In [ ]:
#GRU - BiLSTM , Adam 1e-4, (selu + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model16 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='selu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model16.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',get_f1])

model16.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy16= model16.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy16)

predictions16= model16.predict(x_test)

predictions16=np.argmax(predictions16, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions16)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions16))
print('Recall: %.3f' % recall_score(y_test, predictions16))
print('Precision: %.3f' % precision_score(y_test, predictions16))
print('F1 Score: %.3f' % f1_score(y_test, predictions16))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 15s 18ms/step - loss: 0.4542 - accuracy: 0.8213 - get_f1: 0.8212
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2462 - accuracy: 0.8997 - get_f1: 0.8997
Epoch 3/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1240 - accuracy: 0.9554 - get_f1: 0.9554
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0668 - accuracy: 0.9771 - get_f1: 0.9770
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0420 - accuracy: 0.9871 - get_f1: 0.9871
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0323 - accuracy: 0.9899 - get_f1: 0.9899
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0278 - accuracy: 0.9909 - get_f1: 0.9909
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0251 - accuracy: 0.9922 - get_f1: 0.9922
Epoch 9/10
612/612 [==============================] - 11s 1

In [ ]:
#GRU - BiLSTM , Adam 1e-4, (elu + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model17 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model17.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',get_f1])

model17.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy17= model17.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy17)

predictions17= model17.predict(x_test)

predictions17=np.argmax(predictions17, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions17)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions17))
print('Recall: %.3f' % recall_score(y_test, predictions17))
print('Precision: %.3f' % precision_score(y_test, predictions17))
print('F1 Score: %.3f' % f1_score(y_test, predictions17))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 17s 21ms/step - loss: 0.4625 - accuracy: 0.8187 - get_f1: 0.8187
Epoch 2/10
612/612 [==============================] - 11s 17ms/step - loss: 0.2591 - accuracy: 0.8933 - get_f1: 0.8933
Epoch 3/10
612/612 [==============================] - 10s 17ms/step - loss: 0.1326 - accuracy: 0.9508 - get_f1: 0.9508
Epoch 4/10
612/612 [==============================] - 11s 17ms/step - loss: 0.0749 - accuracy: 0.9745 - get_f1: 0.9745
Epoch 5/10
612/612 [==============================] - 10s 17ms/step - loss: 0.0492 - accuracy: 0.9841 - get_f1: 0.9841
Epoch 6/10
612/612 [==============================] - 10s 17ms/step - loss: 0.0355 - accuracy: 0.9892 - get_f1: 0.9892
Epoch 7/10
612/612 [==============================] - 10s 17ms/step - loss: 0.0314 - accuracy: 0.9908 - get_f1: 0.9908
Epoch 8/10
612/612 [==============================] - 10s 17ms/step - loss: 0.0263 - accuracy: 0.9917 - get_f1: 0.9917
Epoch 9/10
612/612 [==============================] - 10s 1

In [ ]:
#GRU - BiLSTM , Adam 1e-4, (exponential + softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model18 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='exponential'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model18.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',get_f1])

model18.fit(x_train, y_train, batch_size=32, epochs=10)
accuracy18= model18.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy18)

predictions18= model18.predict(x_test)

predictions18=np.argmax(predictions18, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions18)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions18))
print('Recall: %.3f' % recall_score(y_test, predictions18))
print('Precision: %.3f' % precision_score(y_test, predictions18))
print('F1 Score: %.3f' % f1_score(y_test, predictions18))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 17s 18ms/step - loss: 0.6184 - accuracy: 0.7641 - get_f1: 0.7641
Epoch 2/10
612/612 [==============================] - 11s 17ms/step - loss: 0.3534 - accuracy: 0.8640 - get_f1: 0.8640
Epoch 3/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1749 - accuracy: 0.9367 - get_f1: 0.9368
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1002 - accuracy: 0.9676 - get_f1: 0.9676
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0652 - accuracy: 0.9800 - get_f1: 0.9800
Epoch 6/10
612/612 [==============================] - 11s 17ms/step - loss: 0.0470 - accuracy: 0.9869 - get_f1: 0.9869
Epoch 7/10
612/612 [==============================] - 11s 17ms/step - loss: 0.0384 - accuracy: 0.9891 - get_f1: 0.9891
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0323 - accuracy: 0.9900 - get_f1: 0.9900
Epoch 9/10
612/612 [==============================] - 11s 1

# Batch Size Tuning

In [ ]:
# GRU layer followed by BiLSTM layer, Batch Size (10)
from tensorflow.keras import layers
model19 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model19.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model19.fit(x_train, y_train, batch_size=10, epochs=10)

predictions19= model19.predict(x_test)

predictions19=np.argmax(predictions19, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions19))
cm = confusion_matrix(y_test, predictions19)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions19))
print('F1 Score: %.3f' % f1_score(y_test, predictions19))
print('Precision: %.3f' % precision_score(y_test, predictions19))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1958/1958 [==============================] - 43s 17ms/step - loss: 0.4212 - accuracy: 0.8298
Epoch 2/10
1958/1958 [==============================] - 33s 17ms/step - loss: 0.2187 - accuracy: 0.9150
Epoch 3/10
1958/1958 [==============================] - 34s 17ms/step - loss: 0.1165 - accuracy: 0.9610
Epoch 4/10
1958/1958 [==============================] - 34s 17ms/step - loss: 0.0704 - accuracy: 0.9773
Epoch 5/10
1958/1958 [==============================] - 34s 17ms/step - loss: 0.0470 - accuracy: 0.9859
Epoch 6/10
1958/1958 [==============================] - 34s 17ms/step - loss: 0.0383 - accuracy: 0.9885
Epoch 7/10
1958/1958 [==============================] - 34s 17ms/step - loss: 0.0314 - accuracy: 0.9915
Epoch 8/10
1958/1958 [==============================] - 34s 18ms/step - loss: 0.0279 - accuracy: 0.9915
Epoch 9/10
1958/1958 [==============================] - 33s 17ms/step - loss: 0.0235 - accuracy: 0.9925
Epoch 10/10
204/204 [==============================] - 2s 5ms/step
Recall: 

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
# GRU layer followed by BiLSTM layer, Batch Size (40)
from tensorflow.keras import layers
model20 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model20.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model20.fit(x_train, y_train, batch_size=40, epochs=10)

predictions20= model20.predict(x_test)

predictions20=np.argmax(predictions20, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions20))
cm = confusion_matrix(y_test, predictions20)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions20))
print('F1 Score: %.3f' % f1_score(y_test, predictions20))
print('Precision: %.3f' % precision_score(y_test, predictions20))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


490/490 [==============================] - 19s 23ms/step - loss: 0.4815 - accuracy: 0.8174
Epoch 2/10
490/490 [==============================] - 9s 18ms/step - loss: 0.2920 - accuracy: 0.8789
Epoch 3/10
490/490 [==============================] - 9s 17ms/step - loss: 0.1516 - accuracy: 0.9474
Epoch 4/10
490/490 [==============================] - 8s 17ms/step - loss: 0.0898 - accuracy: 0.9706
Epoch 5/10
490/490 [==============================] - 8s 17ms/step - loss: 0.0583 - accuracy: 0.9829
Epoch 6/10
490/490 [==============================] - 8s 17ms/step - loss: 0.0421 - accuracy: 0.9877
Epoch 7/10
490/490 [==============================] - 8s 17ms/step - loss: 0.0345 - accuracy: 0.9897
Epoch 8/10
490/490 [==============================] - 8s 17ms/step - loss: 0.0299 - accuracy: 0.9917
Epoch 9/10
490/490 [==============================] - 8s 17ms/step - loss: 0.0273 - accuracy: 0.9921
Epoch 10/10
204/204 [==============================] - 2s 7ms/step
Recall: 0.681
The Confusion Matrix

In [ ]:
# GRU layer followed by BiLSTM layer, Batch Size (60)
from tensorflow.keras import layers
model21 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model21.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model21.fit(x_train, y_train, batch_size=60, epochs=10)

predictions21= model21.predict(x_test)

predictions21=np.argmax(predictions21, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions21))
cm = confusion_matrix(y_test, predictions21)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions21))
print('F1 Score: %.3f' % f1_score(y_test, predictions21))
print('Precision: %.3f' % precision_score(y_test, predictions21))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


327/327 [==============================] - 15s 25ms/step - loss: 0.4978 - accuracy: 0.8187
Epoch 2/10
327/327 [==============================] - 8s 23ms/step - loss: 0.3457 - accuracy: 0.8531
Epoch 3/10
327/327 [==============================] - 8s 24ms/step - loss: 0.1757 - accuracy: 0.9349
Epoch 4/10
327/327 [==============================] - 6s 17ms/step - loss: 0.1020 - accuracy: 0.9662
Epoch 5/10
327/327 [==============================] - 6s 17ms/step - loss: 0.0650 - accuracy: 0.9804
Epoch 6/10
327/327 [==============================] - 6s 17ms/step - loss: 0.0462 - accuracy: 0.9860
Epoch 7/10
327/327 [==============================] - 6s 17ms/step - loss: 0.0378 - accuracy: 0.9891
Epoch 8/10
327/327 [==============================] - 6s 17ms/step - loss: 0.0321 - accuracy: 0.9912
Epoch 9/10
327/327 [==============================] - 6s 17ms/step - loss: 0.0289 - accuracy: 0.9916
Epoch 10/10
204/204 [==============================] - 2s 5ms/step
Recall: 0.663
The Confusion Matrix

In [ ]:
# GRU layer followed by BiLSTM layer, Batch Size (80)
from tensorflow.keras import layers
model22 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model22.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model22.fit(x_train, y_train, batch_size=80, epochs=10)

predictions22= model22.predict(x_test)

predictions22=np.argmax(predictions22, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions22))
cm = confusion_matrix(y_test, predictions22)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions22))
print('F1 Score: %.3f' % f1_score(y_test, predictions22))
print('Precision: %.3f' % precision_score(y_test, predictions22))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


245/245 [==============================] - 10s 21ms/step - loss: 0.5074 - accuracy: 0.8190
Epoch 2/10
245/245 [==============================] - 5s 22ms/step - loss: 0.3849 - accuracy: 0.8343
Epoch 3/10
245/245 [==============================] - 5s 22ms/step - loss: 0.2096 - accuracy: 0.9194
Epoch 4/10
245/245 [==============================] - 6s 24ms/step - loss: 0.1190 - accuracy: 0.9593
Epoch 5/10
245/245 [==============================] - 4s 18ms/step - loss: 0.0749 - accuracy: 0.9757
Epoch 6/10
245/245 [==============================] - 5s 18ms/step - loss: 0.0520 - accuracy: 0.9840
Epoch 7/10
245/245 [==============================] - 4s 18ms/step - loss: 0.0399 - accuracy: 0.9878
Epoch 8/10
245/245 [==============================] - 4s 18ms/step - loss: 0.0330 - accuracy: 0.9901
Epoch 9/10
245/245 [==============================] - 4s 18ms/step - loss: 0.0292 - accuracy: 0.9912
Epoch 10/10
204/204 [==============================] - 2s 5ms/step
Recall: 0.698
The Confusion Matrix

# Epochs Tuning

In [ ]:
# GRU layer followed by BiLSTM layer, Batch Size (32), Epochs(25)
from tensorflow.keras import layers
model23 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model23.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model23.fit(x_train, y_train, batch_size=32, epochs=25)

predictions23= model23.predict(x_test)

predictions23=np.argmax(predictions23, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions23)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions23))
print('Recall: %.3f' % recall_score(y_test, predictions23))
print('Precision: %.3f' % precision_score(y_test, predictions23))
print('F1 Score: %.3f' % f1_score(y_test, predictions23))

Epoch 1/25


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 18s 20ms/step - loss: 0.4754 - accuracy: 0.8216
Epoch 2/25
612/612 [==============================] - 10s 17ms/step - loss: 0.2930 - accuracy: 0.8791
Epoch 3/25
612/612 [==============================] - 10s 17ms/step - loss: 0.1507 - accuracy: 0.9471
Epoch 4/25
612/612 [==============================] - 13s 21ms/step - loss: 0.0871 - accuracy: 0.9727
Epoch 5/25
612/612 [==============================] - 16s 26ms/step - loss: 0.0588 - accuracy: 0.9823
Epoch 6/25
612/612 [==============================] - 15s 24ms/step - loss: 0.0417 - accuracy: 0.9875
Epoch 7/25
612/612 [==============================] - 11s 18ms/step - loss: 0.0344 - accuracy: 0.9901
Epoch 8/25
612/612 [==============================] - 12s 19ms/step - loss: 0.0296 - accuracy: 0.9911
Epoch 9/25
612/612 [==============================] - 15s 24ms/step - loss: 0.0260 - accuracy: 0.9927
Epoch 10/25
612/612 [==============================] - 13s 21ms/step - loss: 0.0240 - accurac

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
# GRU layer followed by BiLSTM layer, Batch Size (32), Epochs(50)
from tensorflow.keras import layers
model24 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model24.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model24.fit(x_train, y_train, batch_size=32, epochs=50)

predictions24= model24.predict(x_test)

predictions24=np.argmax(predictions24, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions24)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions24))
print('Recall: %.3f' % recall_score(y_test, predictions24))
print('Precision: %.3f' % precision_score(y_test, predictions24))
print('F1 Score: %.3f' % f1_score(y_test, predictions24))

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 14s 17ms/step - loss: 0.4783 - accuracy: 0.8164
Epoch 2/50
612/612 [==============================] - 10s 17ms/step - loss: 0.2788 - accuracy: 0.8891
Epoch 3/50
612/612 [==============================] - 10s 16ms/step - loss: 0.1431 - accuracy: 0.9497
Epoch 4/50
612/612 [==============================] - 10s 17ms/step - loss: 0.0813 - accuracy: 0.9747
Epoch 5/50
612/612 [==============================] - 10s 17ms/step - loss: 0.0533 - accuracy: 0.9845
Epoch 6/50
612/612 [==============================] - 10s 17ms/step - loss: 0.0392 - accuracy: 0.9894
Epoch 7/50
612/612 [==============================] - 11s 17ms/step - loss: 0.0329 - accuracy: 0.9913
Epoch 8/50
612/612 [==============================] - 10s 17ms/step - loss: 0.0296 - accuracy: 0.9917
Epoch 9/50
612/612 [==============================] - 11s 18ms/step - loss: 0.0280 - accuracy: 0.9925
Epoch 10/50
612/612 [==============================] - 10s 16ms/step - loss: 0.0247 - accurac

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
# GRU layer followed by BiLSTM layer, Batch Size (32), Epochs(75)
from tensorflow.keras import layers
model25 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model25.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model25.fit(x_train, y_train, batch_size=32, epochs=75)

predictions25= model25.predict(x_test)

predictions25=np.argmax(predictions25, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions25)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions25))
print('Recall: %.3f' % recall_score(y_test, predictions25))
print('Precision: %.3f' % precision_score(y_test, predictions25))
print('F1 Score: %.3f' % f1_score(y_test, predictions25))

Epoch 1/75


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 14s 17ms/step - loss: 0.4728 - accuracy: 0.8193
Epoch 2/75
612/612 [==============================] - 11s 19ms/step - loss: 0.2736 - accuracy: 0.8871
Epoch 3/75
612/612 [==============================] - 11s 17ms/step - loss: 0.1405 - accuracy: 0.9500
Epoch 4/75
612/612 [==============================] - 11s 17ms/step - loss: 0.0815 - accuracy: 0.9722
Epoch 5/75
612/612 [==============================] - 11s 17ms/step - loss: 0.0538 - accuracy: 0.9838
Epoch 6/75
612/612 [==============================] - 11s 17ms/step - loss: 0.0401 - accuracy: 0.9889
Epoch 7/75
612/612 [==============================] - 11s 17ms/step - loss: 0.0338 - accuracy: 0.9903
Epoch 8/75
612/612 [==============================] - 11s 17ms/step - loss: 0.0288 - accuracy: 0.9922
Epoch 9/75
612/612 [==============================] - 11s 17ms/step - loss: 0.0274 - accuracy: 0.9919
Epoch 10/75
612/612 [==============================] - 11s 17ms/step - loss: 0.0241 - accurac